# Index Fund Arbitrage

## Introduction

An index fund will hold a basket of assets for some sort of general investment thesis. This allows individuals to buy and sell the index fund shares but essentially "own" the basket instead of needing to form the basket themselves.

Every day, an index fund's NAV, Net Asset Value, is calculated and it represents the underlying value of the basket. The actual share price for the index fund shares can be at a premium (higher than the NAV) or at a discount (lower than the NAV). However, we expect an equilibrium over time where there is on average no premium or discount.

This example will track the price of a basket of goods as well as an index fund and try to see what dynamics might arise in terms of potential arbitrage. We will assume there are two types of actors:

1. Arbitrage Actor: This actor brings the price of the index fund in line with the basket of goods through arbitrage.
2. Momentum Trader: This actor will bet on momentum.

## Historical Set Up

Historically, we will say that the basket of goods and the index fund both gain and lose an amount which is highly correlated with one and other. There are also actions which are taken on the index fund. The steps for creating this series of hypothetical data is below:

1. Basket of goods grows in percentage by $r_b = N(\mu_1, \sigma^2_1)$
2. The index fund grows in percentage by $r_i = \lambda r_b + (1-\lambda)\cdot N(\mu_2, \sigma^2_2)$
3. The arbitrage trader brings the index fund in line with the basket of goods if $|P_b - P_i| > \theta_1$
4. The momentum trader will trade with the momentum if the N day return of the basket on an absolute basis is greater than 

In [9]:
import sqlite3
import pandas as pd
con = sqlite3.connect(':memory:')


In [10]:
import numpy as np

In [11]:
np.random.seed(0)

In [ ]:
true_prices = pd.DataFrame()